In [69]:
import numpy as np
import pandas as pd

In [70]:
###################### Funcion calcular sindrome ######################
def calcular_sindrome(patron_error, matriz_Ht):
    sindrome = []
    detectable = []
    for i in patron_error:
        valor = []
        i2 = [np.float64(j) for j in np.array(list(i))]
        for x in range(len(matriz_Ht[0])):
            multiply_ih = i2*matriz_Ht[:,x]
            valid = np.count_nonzero(multiply_ih == 1)
            if valid%2 != 0:
                valor.append(1)
            else:
                valor.append(0)
        sindrome.append(valor)
        if valor == [0, 0, 0]:
            detectable.append("no")
        else:
            detectable.append("si")
    return sindrome, detectable
    

In [71]:
###################### Datos para el ejercicio ######################
n=7
k=4
p_e = 1e-3
matriz_P = np.array([[1,1,0],[0,1,1],[1,1,1],[1,0,1]], dtype=int)
matriz_Ik = np.identity(k,dtype=int)
matriz_G = np.concatenate((matriz_P.T,matriz_Ik.T),axis=0,dtype=int).T
matriz_H = np.concatenate((matriz_Ik[:-1].T[:-1],matriz_P),axis=0,dtype=int).T
matriz_Ht = matriz_H.T
print(matriz_G)

[[1 1 0 1 0 0 0]
 [0 1 1 0 1 0 0]
 [1 1 1 0 0 1 0]
 [1 0 1 0 0 0 1]]


In [72]:
## calculo de patrones de error
patron_error = np.array(["{0:07b}".format(x) for x in range(128)])
cantidad_error = np.array([x.count('1') for x in patron_error])
sindrome, detectable = calcular_sindrome(patron_error,matriz_Ht)
corregibles = list(map(lambda x: "no" if x>1 else "si", cantidad_error))
probabilidad = [p_e**x * (1-p_e)**(n-x) for x in cantidad_error]

In [73]:
calculo_patrones = pd.DataFrame({'patron_error':patron_error,'cantidad_error':cantidad_error,'sindrome':sindrome,'detectable':detectable,'corregibles':corregibles,'probabilidad':probabilidad})
calculo_patrones

,patron_error,cantidad_error,sindrome,detectable,corregibles,probabilidad
0,0000000,0,"[0, 0, 0]",no,si,9.930210e-01
1,0000001,1,"[1, 0, 1]",si,si,9.940150e-04
2,0000010,1,"[1, 1, 1]",si,si,9.940150e-04
3,0000011,2,"[0, 1, 0]",si,no,9.950100e-07
4,0000100,1,"[0, 1, 1]",si,si,9.940150e-04
...,...,...,...,...,...,...
123,1111011,6,"[0, 1, 1]",si,no,9.990000e-19
124,1111100,5,"[0, 1, 0]",si,no,9.980010e-16
125,1111101,6,"[1, 1, 1]",si,no,9.990000e-19
126,1111110,6,"[1, 0, 1]",si,no,9.990000e-19


In [74]:
###################### Sindrome ######################
#np.where((data['sport'] == '0x0000'), '0', data['sport'])
lista_errores_corregibles = []
lista_sindrome = []
for indice in range(len(calculo_patrones)):
    if (calculo_patrones.patron_error[indice].count('1') < 2):
        lista_errores_corregibles.append(calculo_patrones.patron_error[indice])
        lista_sindrome.append(calculo_patrones.sindrome[indice])
tabla_sindrome = pd.DataFrame({'patron_error_corregible':lista_errores_corregibles,'sindrome':lista_sindrome})
tabla_sindrome

,patron_error_corregible,sindrome
0,0000000,"[0, 0, 0]"
1,0000001,"[1, 0, 1]"
2,0000010,"[1, 1, 1]"
3,0000100,"[0, 1, 1]"
4,0001000,"[1, 1, 0]"
5,0010000,"[0, 0, 1]"
6,0100000,"[0, 1, 0]"
7,1000000,"[1, 0, 0]"


In [76]:
###################### Transmisor ######################
f = open ('mensaje_codificado.txt','r')
mensaje = f.read()
while(len(mensaje)%4 != 0):
    mensaje+='0'
mensaje = np.array(list(map(list, zip(*[map(int, list(mensaje))] * k))),dtype=int)
mensaje = mensaje @ matriz_G
mensaje_generadora = [[0 if a%2==0 else 1 for a in x] for x in mensaje]
mensaje_enviar = "".join(["".join([str(ele) for ele in sub]) for sub in mensaje_generadora])
print(mensaje_enviar)
f = open ('mensaje_transmisor.txt','w')
f.write(mensaje_enviar)
f.close()

1110010101000110100010000000110010111100100010111000110100011010010111110100000011010010111011010010100010100011000110111111110100011110010111010000110100101000101110011011100000000000011011100101011100110001101001011110100010100011000110011100101110010010111000000000011010111001111111111001010000000011010011001011000110111001001000111100101010111001000111011100101110001110010010111100011010111000100011110010111001010111001101000101000110000000001011111010001101000110100011100100000000000000011010000110100000000001110010010111110010100000001000110110010100000001011100010001100000001110010011010011100101010001000110110010110000000111001011001011101000110100001110011100101010001111100100010111011010010100010000000101000100110101000110110010110111000011010000000010100010110100000000010001101100101101000100000001010001010001100000000001101001101000101111101000101000101000111011100100011001101000010111110100000011010000000100011000000000000000011010011010000001101010001111010000110100011010

In [85]:
###################### Receptor ######################
f = open ('mensaje_transmisor.txt','r')
mensaje = f.read()
lista_mensaje = []
mensaje_7 = ""
for i in mensaje:
    if len(mensaje_7) < 7:
        mensaje_7 += i
    else:
        lista_mensaje.append(mensaje_7)
        mensaje_7 = ""
sindrome_recepcion, detectable_recepcion = calcular_sindrome(lista_mensaje,matriz_Ht)

In [84]:
calculo_patrones_recepcion = pd.DataFrame({'mensaje_recibido':lista_mensaje,'sindrome':sindrome_recepcion,'detectable':detectable_recepcion})
calculo_patrones_recepcion

,mensaje_recibido,sindrome,detectable
0,1110010,"[0, 0, 0]",no
1,0100011,"[0, 0, 0]",no
2,1000100,"[1, 1, 1]",si
3,0000110,"[1, 0, 0]",si
4,1011110,"[1, 1, 1]",si
...,...,...,...
876,1001010,"[1, 0, 1]",si
877,0110111,"[0, 1, 0]",si
878,1111000,"[0, 0, 1]",si
879,0000000,"[0, 0, 0]",no


In [ ]:
lista_correjibles = list(np.where(calculo_patrones_recepcion.corregibles=="si")[0])
for i in lista_correjibles:
    print(i)
print(calculo_patrones_recepcion.mensaje_recibido.iloc[-1])
sindrome_valor = calculo_patrones_recepcion.sindrome.iloc[-1]
print(type(sindrome_valor))
print(type(tabla_sindrome.sindrome.iloc[0]))
tabla_sindrome.where(tabla_sindrome.sindrome == sindrome_valor)


9
22
37
55
60
61
64
70
89
95
98
101
104
114
116
117
118
128
129
130
135
146
153
164
175
177
181
182
186
192
206
207
208
209
218
234
243
246
250
251
268
280
281
286
295
302
312
318
324
359
360
378
387
404
406
407
441
445
449
455
464
473
482
487
488
491
494
495
496
498
500
512
524
529
530
532
544
552
555
558
579
582
589
590
595
620
629
630
635
639
644
650
657
691
693
748
759
773
774
778
792
798
805
822
847
879
880
0000000
<class 'list'>
<class 'list'>


ValueError: ('Lengths must match to compare', (8,), (3,))